# Some example demographies

We illustrate some advanced options of `momi.Demography()`.

In [1]:
from momi import Demography, to_ms_cmd

### Specifying demographies in different time scales

Users can choose to input demographies in ms time scale, the more 'standard' coalescent time scale, or in per-generation units.

In [2]:
# a simple Demography, in ms units
events = [('-eg', 0.0, 'a', 1.0), ('-ej', .5, 'a', 'b'), ('-en', 1., 'b', .5)]
demo = Demography(events, sampled_pops=('a','b'), sampled_n=(5,3))

print "Demography in ms syntax:"
print to_ms_cmd(demo)

Demography in ms syntax:
-I 2 5 3 -eg 0.0 1 1.0 -ej 0.5 1 2 -en 1.0 2 0.5


In [3]:
# the same Demography, but specified in the 'standard' (not 'ms') time scale
# this means that there are 2*N_ref generations per unit time (instead of 4*N_ref as in ms)

# notice times are 2x longer in the 'standard' units.
# as consequence, growth rate per unit time is 2x slower
events = [('-eg', 0.0, 'a', 0.5), ('-ej', 1., 'a', 'b'), ('-en', 2., 'b', .5)]

# use time_scale='standard' to specify standard coalescent units
demo = Demography(events, sampled_pops=('a','b'), sampled_n=(5,3), time_scale='standard')

try:
    print to_ms_cmd(demo)
except:
    print "Must convert demography to ms units before calling to_ms_cmd()\n"

demo = demo.rescaled()
print "Demography in ms syntax & units:"
print to_ms_cmd(demo)

Must convert demography to ms units before calling to_ms_cmd()

Demography in ms syntax & units:
-I 2 5 3 -eg 0.0 1 1.0 -ej 0.5 1 2 -en 1.0 2 0.5


In [4]:
# the same Demography, but specified in per-generation units
# we'll assume a diploid population size of 10,000

# join event 5e3 generations ago. population size drops to 5000 at 1e4 generations ago.
# growth rate per generation is 1e-4 in population 'a'
events = [('-eg',0.0,'a',1e-4),('-ej', 5e3, 'a', 'b'), ('-en', 1e4, 'b', 5000)]

# use default_N to set the diploid population size
demo = Demography(events, sampled_pops=('a','b'), sampled_n=(5,3), default_N=10000)

try:
    print to_ms_cmd(demo)
except:
    print "Must convert demography to ms units before calling to_ms_cmd()\n"

demo = demo.rescaled()
print "Demography in ms syntax & units:"
print to_ms_cmd(demo)

Must convert demography to ms units before calling to_ms_cmd()

Demography in ms syntax & units:
-I 2 5 3 -eg 0.0 1 1.0 -ej 0.5 1 2 -en 1.0 2 0.5


### Ghost populations

A 'ghost' population is one that is not present in the sample.

`momi` syntax is more flexible than `ms` about ghost populations. In particular, ghost populations can be immediately affected by demographic events.

By contrast, in `ms`, ghost populations cannot be modified by events until they have been "created" with the **-es** flag.

In [5]:
# a demography with one sampled population 'a', and a ghost population 'z'

events = [('-eg', 0.0, 'z', 1.0), # can set growth rate of 'z' at t=0, even tho its a ghost population
          ('-ep', .5, 'a', 'z', .25), # pulse migration from z to a
          ('-ej', 1.0, 'z', 'a')]

demo = Demography(events, ['a'], [10])
print "Demography in ms syntax."
print to_ms_cmd(demo)
# Note in ms syntax, the growth event (-eg) doesn't happen until the admixture event (-es)

Demography in ms syntax.
-I 1 10 -es 0.5 1 0.75 -en 0.5 2 0.606530659713 -eg 0.5 2 1.0 -ej 1.0 2 1


In `momi`, we can also have sampled populations merge **-ej** with ghost populations, even if there is no admixture event.

In [6]:
# a demography with one sampled population 'a', and a ghost population 'z'

events = [('-eg', 0.0, 'z', 1.0),
          ('-ej', .5, 'a', 'z'), # 'z' is empty, so this is equivalent to just changing size/growth of 'a'
          ('-en', 1.0, 'z', 1.0)]

demo = Demography(events, ['a'], [10])
print "Demography in ms syntax."
print to_ms_cmd(demo)
# Note in ms syntax, the -ej event doesn't appear; instead just the size and growth rate of 'a' change

Demography in ms syntax.
-I 1 10 -en 0.5 1 0.606530659713 -eg 0.5 1 1.0 -en 1.0 1 1.0


### Ancient DNA

Use the `sampled_t` parameter to indicate that a population was sampled in the past.

In [7]:
events = [('-ej', 1.0, 'a', 'b')]

# the samples from 'b' are ancient DNA, from time t=0.5
demo = Demography(events, ['a','b'], (6, 10), sampled_t=[0.0, .5])

try:
    print to_ms_cmd(demo)
except:
    print "Cannot convert demography with archaic populations to ms syntax"

Cannot convert demography with archaic populations to ms syntax


### Demography with a lot of parameters

An example on how to programatically construct a demography with a lot of parameters (too many to conveniently write down by hand).

In [8]:
# piecewise constant demography, with 50 pieces

import random

n_pieces = 50
t = 0.0
events = []
for i in range(n_pieces):
    t = t + random.expovariate(100.)
    N = random.expovariate(1.0)
    events.append(('-en', t, 0, N))

demo = Demography(events, [0], [10])
print "Demography in ms syntax."
print to_ms_cmd(demo)

Demography in ms syntax.
-I 1 10 -en 0.00365552903558 1 0.333011892728 -en 0.0125691215032 1 2.92429640904 -en 0.0195182146417 1 2.74991426897 -en 0.0264717104358 1 2.00600073765 -en 0.0298387742432 1 0.0204653656246 -en 0.041327558519 1 1.92790495315 -en 0.0807233837638 1 1.60763155829 -en 0.0823887074074 1 1.17544445502 -en 0.0897952883008 1 0.48169133007 -en 0.100717333328 1 0.393437489202 -en 0.113304822772 1 0.301604204789 -en 0.113454552637 1 0.344336751613 -en 0.120993295857 1 0.84680367143 -en 0.145169816991 1 1.3202257277 -en 0.145478657964 1 0.994010534596 -en 0.147970923738 1 1.80657174337 -en 0.151981467799 1 0.0608316200596 -en 0.152156233246 1 0.762370901568 -en 0.171309221646 1 0.907501845653 -en 0.175198733844 1 4.8738974376 -en 0.177335175876 1 0.224244271015 -en 0.182853727576 1 0.677309957968 -en 0.18370798715 1 0.0509219143499 -en 0.185001352291 1 0.0574347096426 -en 0.185494668622 1 0.848115507801 -en 0.197965904674 1 5.55847738212 -en 0.202775799448 1 0.4741904330